In [78]:
import socket, time, types, select
from collections import namedtuple
from heapq import heappush, heappop
ScheduleEvent = namedtuple('ScheduleEvent', ['event_time', 'task'])
Session = namedtuple('Session', ['address', 'file'])

In [79]:
events = []
sessions = {}
callback = {}
generators = {}

Exception ignored in: <coroutine object nbcaser at 0x7f555e42b2b0>
Traceback (most recent call last):
  File "<ipython-input-59-1413b6cabf9d>", line 26, in nbcaser
KeyError: <socket.socket fd=59, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 9600), raddr=('127.0.0.1', 38650)>


In [80]:
def reactor(host="localhost", port=9600):
    "Main event loop that triggers the appropriate business logic callbacks"
    s = socket.socket()
    s.bind((host, port))
    s.listen(5)
    s.setblocking(0) # Mack async, Never wait on a client socket.
    sessions[s] = None
    print(sessions)
    print('Server up, running, and waiting for call on %s %s'%(host, port))
    try:
        while True:
            ready_to_read, _, _ = select.select(sessions, [], [], 0.1)
            for c in ready_to_read:
                print(c)
                if c is s:
                    c, a = c.accept()
                    print(c, a)
                    connect(c, a)
                    continue
                line = sessions[c].file.readline()
                if line:
                    callback[c](c, line.rstrip())
                else:
                    disconnect(c)
            # Run events scheduled at the appropriate event time
            while events and events[0].event_time <= time.monotonic():
                event = heappop(events)
                event.task()
    finally:
        s.close() 

In [81]:
def connect(c, a):
    'Reactor logic for new connections'
    sessions[c] = Session(a, c.makefile())
    on_connect(c)     

In [82]:
def disconnect(c):
    'Reactor logic to end sessions'
    on_disconnect(c)                         # call into user's business logic
    sessions[c].file.close()
    c.close()
    del sessions[c]
    del callback[c]

In [83]:
def add_task(event_time, task):
    'Helper function to schedule one-time tasks at specific time'
    heappush(events, ScheduleEvent(event_time, task))

In [84]:
def call_later(delay, task):
    'Helper function to schedule one-time tasks after a given delay'
    add_task(time.time() + delay, task)

In [85]:
def call_periodic(delay, interval, task):
    'Helper function to schedule recurring tasks'
    def inner():
        task()
        call_later(interval, inner)
    call_later(delay, inner)

In [86]:
def on_connect(c):
        g = nbcaser(c)          # 'g' is a coroutine
        generators[c] = g       # generators -> awaitables
        callback[c] = g.send(None)  # we do this to advance `nbcaser` coroutine
                                    # to yield through the 'readline' coroutine
                                    # which will sleep on its 'yield' expression

In [87]:
def on_disconnect(c):
        g = generators.pop(c)
        g.close()

In [88]:
@types.coroutine
def readline(c):
    'A non-blocking readline to use with two-way generators'
    def inner(c, line):
        g = generators[c]
        try:
            callback[c] = g.send(line)  # `g.send(line)` will resume the `yield inner` point
        except StopIteration:
            disconnect(c)
    line = yield inner
    return line


In [89]:
def sleep(c, delay):
    'A non-blocking sleep to use with two-way generators'
    def inner():
        g = generators[c]
        callback[c] = next(g)
    call_later(delay, inner)
    return lambda *args: callback[c]

In [90]:
def announcement():
    print('The event loop is still running at:', time.ctime())

call_periodic(delay=1, interval=15, task=announcement)

In [91]:
async def nbcaser(c):
    upper, title = 'upper', 'title'
    mode = upper
    print("Received connection from", sessions[c].address)
    try:
        c.sendall(b'<welcome: starting in upper case mode>\n')
        while 1:
            line = await readline(c)
            if line == 'quit':
                c.sendall(b'quit\r\n')
                return
            if mode is upper and line == 'title':
                c.sendall(b'<switching to title case mode>\r\n')
                mode = title
                continue
            if mode is title and line == 'upper':
                line = c.sendall(b'<switching to upper case mode>\r\n')
                mode = upper
                continue
            print(sessions[c].address, '-->', line)
            if mode is upper:
                c.sendall(b'Upper-cased: %a\r\n' % line.upper())
            else:
                c.sendall(b'Title-cased: %a\r\n' % line.title())
    finally:
        print(sessions[c].address, 'quit')

In [94]:
reactor('localhost', 9613)

{<socket.socket [closed] fd=-1, family=AddressFamily.AF_INET, type=2049, proto=0>: None, <socket.socket fd=63, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 9612)>: Session(address=('127.0.0.1', 33038), file=<_io.TextIOWrapper name=63 mode='r' encoding='UTF-8'>), <socket.socket [closed] fd=-1, family=AddressFamily.AF_INET, type=2049, proto=0>: None, <socket.socket fd=62, family=AddressFamily.AF_INET, type=2049, proto=0, laddr=('127.0.0.1', 9613)>: None}
Server up, running, and waiting for call on localhost 9613


ValueError: file descriptor cannot be a negative integer (-1)

In [118]:
def numTrees(n):
    G = [0] * (n + 1)
    G[0], G[1] = 1, 1
    for m in range(2,n+1):
        G[m] = sum([G[i-1]*G[m-i] for i in range(1, m+1)])
    return G[-1]

In [120]:
numTrees(6)

132

In [121]:
def add(a, b):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-121-1165e286cffd>, line 2)

In [142]:
(1 & 5) << 1

2

In [144]:
1 ^ 5

4

In [145]:
01 101  


SyntaxError: invalid token (<ipython-input-145-fcd4849d5331>, line 1)

In [166]:
(3 & 5)

1

In [155]:
def count_one(n):
    count = 0
    while n:
        n = n&(n-1)
        count += 1
    return count

In [164]:
count_one(4)

1

The set S originally contains numbers from 1 to n. But unfortunately, due to the data error, one of the numbers in the set got duplicated to another number in the set, which results in repetition of one number and loss of another number.

Given an array nums representing the data status of this set after the error. Your task is to firstly find the number occurs twice and then find the number that is missing. Return them in the form of an array.

Example 1:
Input: nums = [1,2,2,4]
Output: [2,3]
Note:
The given array size will in the range [2, 10000].
The given array's numbers won't have any order.